In [1]:
import pandas as pd
import numpy as np

import json
import requests
import folium

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following p

In [ ]:
url = 'http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson'

json_data = requests.get(url).json()['features']
json_data


In [5]:
neighborhood_list=[]
for v in range(len(json_data)):
    
    neighborhood_list.append((
     json_data[v]['properties']['metadata']['region'],#region
     json_data[v]['properties']['name'],#neighborhood
     np.mean(json_data[v]['geometry']['coordinates'][0][0][0][1]),
     np.mean(json_data[v]['geometry']['coordinates'][0][0][0][0])))
  
neighborhood_list

[('antelope-valley', 'Acton', 34.53902300000013, -118.20703400000012),
 ('south-la', 'Adams-Normandie', 34.037396, -118.30800200000012),
 ('santa-monica-mountains',
  'Agoura Hills',
  34.168157000000114,
  -118.77621200000014),
 ('northwest-county', 'Agua Dulce', 34.4881090000001, -118.37822400000013),
 ('san-gabriel-valley', 'Alhambra', 34.10504, -118.12174700000018),
 ('beach-cities',
  'Aliso and Wood Regional Park',
  33.58759300000011,
  -117.75272299999989),
 ('south-county', 'Aliso Viejo', 33.60006400000017, -117.734393),
 ('south-bay', 'Alondra Park', 33.89757200000013, -118.32651300000016),
 ('verdugos', 'Altadena', 34.215508000000106, -118.15135400000013),
 ('north-county', 'Anaheim', 33.8705690000001, -117.74465199999989),
 ('angeles-forest', 'Angeles Crest', 34.473482000000104, -118.09667000000019),
 ('san-gabriel-valley', 'Arcadia', 34.177182, -118.01705200000018),
 ('san-fernando-valley', 'Arleta', 34.264151, -118.4425600000002),
 ('central-la', 'Arlington Heights', 34.0

In [9]:
LA_neighborhood_data = pd.DataFrame(neighborhood_list)
LA_neighborhood_data.columns=['Region','Neighborhood','Latitude','Longitude']
#print("Records with neighborhood data missing:{}".format(LA_neighborhood_data[LA_neighborhood_data['Neighborhood'].isnull()==True]))


#print("Records with region data missing:{}".format(LA_neighborhood_data[LA_neighborhood_data['Region'].isnull()==True]))

LA_neighborhood_data.head()

,Region,Neighborhood,Latitude,Longitude
0,antelope-valley,Acton,34.539023,-118.207034
1,south-la,Adams-Normandie,34.037396,-118.308002
2,santa-monica-mountains,Agoura Hills,34.168157,-118.776212
3,northwest-county,Agua Dulce,34.488109,-118.378224
4,san-gabriel-valley,Alhambra,34.105040,-118.121747


In [7]:
LA_neighborhood_data_grpd=LA_neighborhood_data.groupby('Region',as_index=False).count()
LA_neighborhood_data_grpd

#LA_neighborhood_data['Latitude'].min()

LA_neighborhood_data.head()

,Region,Neighborhood,Latitude,Longitude
0,antelope-valley,Acton,34.539023,-118.207034
1,south-la,Adams-Normandie,34.037396,-118.308002
2,santa-monica-mountains,Agoura Hills,34.168157,-118.776212
3,northwest-county,Agua Dulce,34.488109,-118.378224
4,san-gabriel-valley,Alhambra,34.105040,-118.121747


,Region,Neighborhood,Latitude,Longitude


In [10]:
address = ' los angeles city, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of los angeles City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of los angeles City are 33.9255823, -118.2358324.


In [16]:
# create map of New York using latitude and longitude values
map_LA = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, region, neighborhood in zip(LA_neighborhood_data['Latitude'], LA_neighborhood_data['Longitude'], LA_neighborhood_data['Region'], LA_neighborhood_data['Neighborhood']):
    
    #print(lat,lng)
    label = '{}, {}'.format(neighborhood, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA)  
    
map_LA

In [19]:
central_LA=LA_neighborhood_data[LA_neighborhood_data['Region']=='central-la'].reset_index(drop=True)
central_LA



,Region,Neighborhood,Latitude,Longitude
0,central-la,Arlington Heights,34.052504,-118.316715
1,central-la,Beverly Grove,34.085755,-118.372486
2,central-la,Carthay,34.051684,-118.366636
3,central-la,Chinatown,34.067573,-118.224722
4,central-la,Downtown,34.061247,-118.244623
5,central-la,East Hollywood,34.098356,-118.287392
6,central-la,Echo Park,34.095686,-118.244861
7,central-la,Elysian Park,34.093903,-118.242446
8,central-la,Elysian Valley,34.108424,-118.249729
9,central-la,Fairfax,34.088949,-118.351116


In [27]:
address = 'central los angeles Downtown, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of central los angeles City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of central los angeles City are 34.4987133, -118.5843067.


In [28]:
# create map of New York using latitude and longitude values
map_LA_central= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,  neighborhood in zip(central_LA['Latitude'], central_LA['Longitude'], central_LA['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_LA_central)  
    
map_LA_central

In [10]:
CLIENT_ID = 'QJ3LCZ30QJFWZGOFIDWMN5MTLB0DANIGIWOGT3VE3JXHNGL0' # your Foursquare ID
CLIENT_SECRET = 'FTYTSLO110IBMENPFOCH53BNUHUM53XP4UCTQDVS3YRF4LGR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJ3LCZ30QJFWZGOFIDWMN5MTLB0DANIGIWOGT3VE3JXHNGL0
CLIENT_SECRET:FTYTSLO110IBMENPFOCH53BNUHUM53XP4UCTQDVS3YRF4LGR


In [11]:
url_req="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,'34.10524400000011' ,'-118.32686300000017','Restaurant')
                                                                                                                           
results=requests.get(url_req).json()     
results

{'meta': {'code': 200, 'requestId': '5dc6535b02a172002835c487'},
 'response': {'venues': [{'id': '52c77412498ee9acc1e90889',
    'name': 'Liwan Restaurant & Hookah Lounge',
    'location': {'address': '1783 Westwood Blvd',
     'lat': 34.05035416875937,
     'lng': -118.43777931988191,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.05035416875937,
       'lng': -118.43777931988191}],
     'distance': 11913,
     'postalCode': '90024',
     'cc': 'US',
     'city': 'Los Angeles',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['1783 Westwood Blvd',
      'Los Angeles, CA 90024',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1c0941735',
      'name': 'Mediterranean Restaurant',
      'pluralName': 'Mediterranean Restaurants',
      'shortName': 'Mediterranean',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '7

In [16]:
    url_req="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,'34.10524400000011' ,'-118.32686300000017','Restaurant')
                                                                                                                           
    results=requests.get(url_req).json()     




    venues=results['response']['venues']
    venuelat= venues[0]['location']['lat']
    venuelat


34.05035416875937

In [17]:
def get_venue(neighborlist,inputlatlist,inputlonglist):
 
 venue_detail=[]
 for neighbor,inputlat,inputlong in zip(neighborlist,inputlatlist,inputlonglist):
    print(neighbor,inputlat,inputlong)
    
    url_req="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{},&query={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,inputlat,inputlong,'Restaurant')
                                                                                                                           
    results=requests.get(url_req).json()     
    results




    venues=results['response']['venues']
    #print( venues[1]['delivery']['provider']['name'])
    for i in range(len(results['response']['venues'])):
    
        if('delivery'  in venues[i]):
           venue_detail.append((
           neighbor,
           inputlat,
           inputlong,
           venues[i]['id'],
           venues[i]['name'],
           venues[i]['location']['lat'],
           venues[i]['location']['lng'],
           venues[i]['categories'][0]['name'],
           venues[i]['delivery']['provider']['name']
           ))
        else:
           venue_detail.append((
           neighbor,
           inputlat,
           inputlong,
           venues[i]['id'],
           venues[i]['name'],
           venues[i]['location']['lat'],
           venues[i]['location']['lng'],
           venues[i]['categories'][0]['name'],
           'No Delivery option'
           ))
    

 La_venues=pd.DataFrame(venue_detail)
 La_venues.columns=['Neighborhood','Latitude','Longitude','Venueid','venuename','venueLatitude','venueLongitude','venuecategory','deliveryoption']
 La_venues
    
 return (La_venues)

In [20]:
central_LA_filtered=central_LA[central_LA['Neighborhood'].isin(['Downtown','Chinatown','Westlake','Pico-Union','Arlington Heights','Harvard Heights'])].reset_index(drop=True)

#or (central_LA['Neighborhood']=='Chinatown')

central_LA_filtered

,Region,Neighborhood,Latitude,Longitude
0,central-la,Arlington Heights,34.052504,-118.316715
1,central-la,Chinatown,34.067573,-118.224722
2,central-la,Downtown,34.061247,-118.244623
3,central-la,Harvard Heights,34.052820,-118.313294
4,central-la,Pico-Union,34.052497,-118.281446
5,central-la,Westlake,34.053889,-118.259068


In [21]:
central_la_venues = get_venue(neighborlist=central_LA_filtered['Neighborhood'],
                                   inputlatlist=central_LA_filtered['Latitude'],
                                   inputlonglist=central_LA_filtered['Longitude']
                                  )

print(len(central_la_venues))

central_la_venues.head(40)

#central_la_venues_grouped=central_la_venues.groupby('Neighborhood')

#get_venue(neighborlist,inputlatlist,inputlonglist):

Arlington Heights 34.052504 -118.3167150000001
Chinatown 34.067573 -118.22472200000018
Downtown 34.061247 -118.2446230000002
Harvard Heights 34.052820000000125 -118.31329400000013
Pico-Union 34.05249700000014 -118.28144600000013
Westlake 34.05388900000012 -118.25906800000017
180


,Neighborhood,Latitude,Longitude,Venueid,venuename,venueLatitude,venueLongitude,venuecategory,deliveryoption
0,Arlington Heights,34.052504,-118.316715,52c77412498ee9acc1e90889,Liwan Restaurant & Hookah Lounge,34.050354,-118.437779,Mediterranean Restaurant,grubhub
1,Arlington Heights,34.052504,-118.316715,4c00d9439cf52d7f636714e7,Hae Jang Chon Korean BBQ Restaurant,34.063888,-118.306075,Korean Restaurant,No Delivery option
2,Arlington Heights,34.052504,-118.316715,4aff640ef964a520173822e3,Young Kyung Restaurant,34.052458,-118.303320,Chinese Restaurant,No Delivery option
3,Arlington Heights,34.052504,-118.316715,4c7435bd2db5236a2eadb979,All Family Restaurant,34.053671,-118.323318,Korean Restaurant,No Delivery option
4,Arlington Heights,34.052504,-118.316715,4ade63fcf964a5208f7521e3,Chung Kiwa Restaurant,34.052739,-118.316130,Korean Restaurant,No Delivery option
5,Arlington Heights,34.052504,-118.316715,4b4501a3f964a5200c0126e3,Mi Rak Korean Restaurant 미락,34.049766,-118.308447,Korean Restaurant,No Delivery option
6,Arlington Heights,34.052504,-118.316715,4c7177e1344437042a07275f,Nuevo Santaneco Restaurant,34.046931,-118.308123,Latin American Restaurant,No Delivery option
7,Arlington Heights,34.052504,-118.316715,4c02cb6b0d0e0f47bdaa019a,V.I.P. CHINESE RESTAURANT국빈반점,34.059456,-118.309463,Chinese Restaurant,No Delivery option
8,Arlington Heights,34.052504,-118.316715,4c748327db52b1f7ac7676dc,Olocuita Restaurant,34.063633,-118.309474,Latin American Restaurant,No Delivery option
9,Arlington Heights,34.052504,-118.316715,4ada5935f964a520aa2121e3,El Cholo Restaurant,34.050183,-118.309141,Mexican Restaurant,grubhub


In [143]:
restaurant_category=pd.get_dummies(central_la_venues[['venuecategory','deliveryoption']],prefix="",prefix_sep="")

# add neighborhood column back to dataframe
restaurant_category['Venueid'] = central_la_venues['Venueid'] 

restaurant_category.head()

# move neighborhood column to the first column
fixed_columns = [restaurant_category.columns[-1]] + list(restaurant_category.columns[:-1])
restaurant_category = restaurant_category[fixed_columns]

restaurant_category

,Venueid,American Restaurant,Breakfast Spot,Building,Cajun / Creole Restaurant,Chinese Restaurant,College Cafeteria,Deli / Bodega,Dim Sum Restaurant,Diner,...,Salad Place,Seafood Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Vietnamese Restaurant,Winery,No Delivery option,grubhub
0,52c77412498ee9acc1e90889,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,4c00d9439cf52d7f636714e7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4aff640ef964a520173822e3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4c7435bd2db5236a2eadb979,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4ade63fcf964a5208f7521e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,4aff640ef964a520173822e3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
176,4a29d1fef964a520e2951fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
177,4ad930fcf964a520f61821e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
178,3fd66200f964a520f3ee1ee3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [144]:

restaurant_category_grpd = restaurant_category.groupby('Venueid').mean().reset_index()
restaurant_category_grpd

,Venueid,American Restaurant,Breakfast Spot,Building,Cajun / Creole Restaurant,Chinese Restaurant,College Cafeteria,Deli / Bodega,Dim Sum Restaurant,Diner,...,Salad Place,Seafood Restaurant,Steakhouse,Sushi Restaurant,Taco Place,Thai Restaurant,Vietnamese Restaurant,Winery,No Delivery option,grubhub
0,3fd66200f964a520f3ee1ee3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,43d21a6cf964a5200a2e1fe3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,43ee0ffef964a5205e2f1fe3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,4415b1c5f964a520fd301fe3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4432a229f964a520f3311fe3,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,5a8f4afdaa6c9501448a5314,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
82,5ac505ee8e886a6504090dff,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
83,5afb74bc295513002c74167e,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
84,5c198dad149946002d66b106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [148]:
# set number of clusters
kclusters = 15

category_clustering = restaurant_category.drop('Venueid', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(category_clustering)

# check cluster labels generated for each row in the dataframe
labels=kmeans.labels_
len(labels)

180

In [167]:
central_la_venues['labels']=labels
venue_merged=central_la_venues

#venue_merged = venue_merged.join(central_la_venues.set_index('Neighborhood'), on='Neighbourhood')

venue_merged

venue_merged.groupby('labels').count()

cluster1=venue_merged[venue_merged['labels']==1]
cluster0=venue_merged[venue_merged['labels']==0]
cluster2=venue_merged[venue_merged['labels']==2]
cluster3=venue_merged[venue_merged['labels']==3]
cluster4=venue_merged[venue_merged['labels']==4]
cluster5=venue_merged[venue_merged['labels']==5]
cluster6=venue_merged[venue_merged['labels']==6]
cluster7=venue_merged[venue_merged['labels']==7]
cluster8=venue_merged[venue_merged['labels']==8]
cluster9=venue_merged[venue_merged['labels']==9]
cluster10=venue_merged[venue_merged['labels']==10]
cluster11=venue_merged[venue_merged['labels']==11]
cluster12=venue_merged[venue_merged['labels']==12]
cluster13=venue_merged[venue_merged['labels']==13]
cluster14=venue_merged[venue_merged['labels']==14]


cluster14


,Neighborhood,Latitude,Longitude,Venueid,venuename,venueLatitude,venueLongitude,venuecategory,deliveryoption,labels
26,Arlington Heights,34.052504,-118.316715,49f5ea75f964a520d26b1fe3,NORMS Restaurant,34.079968,-118.376428,Diner,grubhub,14
57,Chinatown,34.067573,-118.224722,43ee0ffef964a5205e2f1fe3,Astro Family Restaurant,34.104606,-118.258843,Diner,grubhub,14
115,Harvard Heights,34.052820,-118.313294,49f5ea75f964a520d26b1fe3,NORMS Restaurant,34.079968,-118.376428,Diner,grubhub,14


In [155]:
venue_merged.groupby(['Venueid','Neighborhood']).count()

Latitude  Longitude  venuename  \
Venueid                  Neighborhood                                        
3fd66200f964a520f3ee1ee3 Arlington Heights         1          1          1   
                         Chinatown                 1          1          1   
                         Downtown                  1          1          1   
                         Harvard Heights           1          1          1   
                         Pico-Union                1          1          1   
...                                              ...        ...        ...   
5ac505ee8e886a6504090dff Chinatown                 1          1          1   
5afb74bc295513002c74167e Chinatown                 1          1          1   
5c198dad149946002d66b106 Chinatown                 1          1          1   
                         Downtown                  1          1          1   
5c708ab5aa6c950039227d59 Downtown                  1          1          1   

                                            venueLatitude  venueLongitude  \
Venueid                  Neighborhood                                       
3fd66200f964a520f3ee1ee3 Arlington Heights              1               1   
                         Chinatown                      1               1   
                         Downtown                       1               1   
                         Harvard Heights                1               1   
                         Pico-Union                     1               1   
...                                                   ...             ...   
5ac505ee8e886a6504090dff Chinatown                      1               1   
5afb74bc295513002c74167e Chinatown                      1               1   
5c198dad149946002d66b106 Chinatown                      1               1   
                         Downtown                       1               1   
5c708ab5aa6c950039227d59 Downtown                       1               1   

                                            venuecategory  deliveryoption  \
Venueid                  Neighborhood                                       
3fd66200f964a520f3ee1ee3 Arlington Heights              1               1   
                         Chinatown                      1               1   
                         Downtown                       1               1   
                         Harvard Heights                1               1   
                         Pico-Union                     1               1   
...                                                   ...             ...   
5ac505ee8e886a6504090dff Chinatown                      1               1   
5afb74bc295513002c74167e Chinatown                      1               1   
5c198dad149946002d66b106 Chinatown                      1               1   
                         Downtown                       1               1   
5c708ab5aa6c950039227d59 Downtown                       1               1   

                                            labels  
Venueid                  Neighborhood               
3fd66200f964a520f3ee1ee3 Arlington Heights       1  
                         Chinatown               1  
                         Downtown                1  
                         Harvard Heights         1  
                         Pico-Union              1  
...                                            ...  
5ac505ee8e886a6504090dff Chinatown               1  
5afb74bc295513002c74167e Chinatown               1  
5c198dad149946002d66b106 Chinatown               1  
                         Downtown                1  
5c708ab5aa6c950039227d59 Downtown                1  

[180 rows x 8 columns]